## Лабораторная работа No1. Python и Pandas.

### Задание 1. 

In [161]:
import pandas as pd

In [162]:
transactions = pd.read_csv('transactions.csv', sep=';', decimal=',')
merchants = pd.read_csv('merchants.csv', sep=';', decimal=',')

In [163]:
merged_df = pd.merge(transactions, merchants, on='merchantid')
# merged_df

In [164]:
merged_df = merged_df[merged_df['fraudbits'] == 0]
# merged_df.info()

In [165]:
guild = merged_df['guild'].value_counts()
print(guild)

guild
Гильдия Обработчиков Шерсти         23337
Ассоциация Оливковых Торговцев      13689
Гильдия Пекарей                     11507
Гильдия Гончаров                    10780
Гильдия Парфюмеров                   9212
Гильдия Августовских Торговцев       8862
Братство Кузнецов                    6820
Братство Виноделов                   3898
Лига Керамистов                      3311
Объединение Фруктовых Садовников     2540
Гильдия Мукомолов                     905
Лига Стеклодувов                      891
Объединение Рыбаков                   775
Союз Судостроителей                   650
Союз Каменотесов                      619
Братство Поставщиков Специй           462
Союз Скотоводов                       315
Общество Ювелиров                     146
Name: count, dtype: int64


In [166]:
# Добавление столбца с амнистией
merged_df['amnesty'] = abs(merged_df['cost'] - merged_df['model_cost']) / merged_df['coef']

merged_df


,citizenid,cost,countertype,date,fraudbits,logid,marketid,merchantid,model_cost,coef,guild,amnesty
0,8315932911717591684,7.39,buy,924-09-27 12:57,0,5694353424282632895,203,100299578,30.0,0.46,Гильдия Парфюмеров,49.152174
1,831575341702446221,22.47,sell,924-09-27 15:48,0,5697043883537282758,1682513,445135,80.0,0.57,Ассоциация Оливковых Торговцев,100.929825
2,8316302311724840691,9.81,buy,924-09-27 18:51,0,5699920682275914112,1820486,57512,690.0,0.86,Гильдия Обработчиков Шерсти,790.918605
3,8316277211719765682,6.21,buy,924-09-27 20:05,0,5701086315294310757,203,90356927,32.5,0.42,Гильдия Обработчиков Шерсти,62.595238
4,5107559044524868241,41.04,buy,924-09-27 05:55,0,5687717990978109338,183,64197274,450.0,0.50,Ассоциация Оливковых Торговцев,817.920000
...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2580876751714134550,3.49,buy,924-09-27 12:55,0,5694330009629049234,2,300913725,700.0,0.69,Гильдия Парфюмеров,1009.434783
99995,5924352725499828772,1.21,buy,924-09-27 05:11,0,5687027643975287047,203,7527417,40.0,0.91,Гильдия Обработчиков Шерсти,42.626374
99996,4189309751670694249,39.11,sell,924-09-27 20:57,0,5701904165786091886,7885278,95400429,45.0,0.16,Союз Судостроителей,36.812500
99997,1198308631680924130,22.84,buy,924-09-27 12:56,0,5694344265869116002,2,36511558,650.0,0.54,Гильдия Гончаров,1161.407407


In [167]:
# Группировка данных по гильдиям для вычисления соотношения продаж к покупкам и средней амнистии
guild_stats = (
    merged_df.groupby('guild')
    .agg(
        sale_count=('countertype', lambda x: (x == 'sell').sum()),
        buy_count=('countertype', lambda x: (x == 'buy').sum()),
        total_amnesty=('amnesty', 'mean'),
    )
    .reset_index()
)

guild_stats

,guild,sale_count,buy_count,total_amnesty
0,Ассоциация Оливковых Торговцев,1330,12359,682.928338
1,Братство Виноделов,392,3506,1726.452617
2,Братство Кузнецов,659,6161,1835.734533
3,Братство Поставщиков Специй,47,415,555.012089
4,Гильдия Августовских Торговцев,849,8013,940.918641
5,Гильдия Гончаров,1019,9761,677.369341
6,Гильдия Мукомолов,78,827,447.359102
7,Гильдия Обработчиков Шерсти,2262,21075,1353.118922
8,Гильдия Парфюмеров,872,8340,926.507174
9,Гильдия Пекарей,1174,10333,1026.286880


In [168]:
# Избегание деления на ноль при расчете соотношения продаж к покупкам
guild_stats['sale_buy_ratio'] = guild_stats['sale_count'] / guild_stats['buy_count'].replace(0, 1)

# Вычисление эффективности гильдий
guild_stats['efficiency'] = guild_stats['sale_buy_ratio'] * guild_stats['total_amnesty']

# Нахождение гильдии с максимальной эффективностью
best_guild = guild_stats.loc[guild_stats['efficiency'].idxmax()]
best_guild_name = best_guild['guild']
best_guild_efficiency = best_guild['efficiency']

print(f"Гильдия с наибольшей эффективностью: {best_guild_name}")
print(f"Эффективность: {best_guild_efficiency:.2f}")

Гильдия с наибольшей эффективностью: Братство Кузнецов
Эффективность: 196.36


In [169]:
# Определение часа с наибольшим количеством продаж для лучшей гильдии
merged_df['date'] = '1' + merged_df['date']
merged_df


,citizenid,cost,countertype,date,fraudbits,logid,marketid,merchantid,model_cost,coef,guild,amnesty
0,8315932911717591684,7.39,buy,1924-09-27 12:57,0,5694353424282632895,203,100299578,30.0,0.46,Гильдия Парфюмеров,49.152174
1,831575341702446221,22.47,sell,1924-09-27 15:48,0,5697043883537282758,1682513,445135,80.0,0.57,Ассоциация Оливковых Торговцев,100.929825
2,8316302311724840691,9.81,buy,1924-09-27 18:51,0,5699920682275914112,1820486,57512,690.0,0.86,Гильдия Обработчиков Шерсти,790.918605
3,8316277211719765682,6.21,buy,1924-09-27 20:05,0,5701086315294310757,203,90356927,32.5,0.42,Гильдия Обработчиков Шерсти,62.595238
4,5107559044524868241,41.04,buy,1924-09-27 05:55,0,5687717990978109338,183,64197274,450.0,0.50,Ассоциация Оливковых Торговцев,817.920000
...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2580876751714134550,3.49,buy,1924-09-27 12:55,0,5694330009629049234,2,300913725,700.0,0.69,Гильдия Парфюмеров,1009.434783
99995,5924352725499828772,1.21,buy,1924-09-27 05:11,0,5687027643975287047,203,7527417,40.0,0.91,Гильдия Обработчиков Шерсти,42.626374
99996,4189309751670694249,39.11,sell,1924-09-27 20:57,0,5701904165786091886,7885278,95400429,45.0,0.16,Союз Судостроителей,36.812500
99997,1198308631680924130,22.84,buy,1924-09-27 12:56,0,5694344265869116002,2,36511558,650.0,0.54,Гильдия Гончаров,1161.407407


In [170]:

# Преобразование столбца date в формат datetime
merged_df['date'] = pd.to_datetime(merged_df['date'], format="%Y-%m-%d %H:%M")
merged_df['hour'] = merged_df['date'].dt.hour

merged_df

,citizenid,cost,countertype,date,fraudbits,logid,marketid,merchantid,model_cost,coef,guild,amnesty,hour
0,8315932911717591684,7.39,buy,1924-09-27 12:57:00,0,5694353424282632895,203,100299578,30.0,0.46,Гильдия Парфюмеров,49.152174,12
1,831575341702446221,22.47,sell,1924-09-27 15:48:00,0,5697043883537282758,1682513,445135,80.0,0.57,Ассоциация Оливковых Торговцев,100.929825,15
2,8316302311724840691,9.81,buy,1924-09-27 18:51:00,0,5699920682275914112,1820486,57512,690.0,0.86,Гильдия Обработчиков Шерсти,790.918605,18
3,8316277211719765682,6.21,buy,1924-09-27 20:05:00,0,5701086315294310757,203,90356927,32.5,0.42,Гильдия Обработчиков Шерсти,62.595238,20
4,5107559044524868241,41.04,buy,1924-09-27 05:55:00,0,5687717990978109338,183,64197274,450.0,0.50,Ассоциация Оливковых Торговцев,817.920000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2580876751714134550,3.49,buy,1924-09-27 12:55:00,0,5694330009629049234,2,300913725,700.0,0.69,Гильдия Парфюмеров,1009.434783,12
99995,5924352725499828772,1.21,buy,1924-09-27 05:11:00,0,5687027643975287047,203,7527417,40.0,0.91,Гильдия Обработчиков Шерсти,42.626374,5
99996,4189309751670694249,39.11,sell,1924-09-27 20:57:00,0,5701904165786091886,7885278,95400429,45.0,0.16,Союз Судостроителей,36.812500,20
99997,1198308631680924130,22.84,buy,1924-09-27 12:56:00,0,5694344265869116002,2,36511558,650.0,0.54,Гильдия Гончаров,1161.407407,12


In [171]:
# Фильтрация данных для лучшей гильдии и только продаж
best_guild_sales = merged_df[(merged_df['guild'] == best_guild_name) & (merged_df['countertype'] == 'sell')]

# Группировка данных по часам для подсчета количества продаж
hourly_sales = best_guild_sales.groupby('hour').size().reset_index(name='sales_count')
hourly_sales

,hour,sales_count
0,0,15
1,1,4
2,2,6
3,3,6
4,4,5
5,5,12
6,6,17
7,7,34
8,8,38
9,9,30


In [172]:

# Нахождение часа с наибольшим количеством продаж
best_hour = hourly_sales.loc[hourly_sales['sales_count'].idxmax()]['hour']

print(f"Час с наибольшим количеством продаж для гильдии {best_guild_name}: {best_hour}")

Час с наибольшим количеством продаж для гильдии Братство Кузнецов: 21


### Задание 2.